In [3]:
import numpy as np
import pandas as pd
import utils
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [4]:
train, test = pd.read_csv('data/train.csv'), pd.read_csv('data/test.csv')
cat_cols = [f for f in train.columns if 'cat' in f]
cont_cols = [f for f in test.columns if 'cont' in f]

#Merge the dataframe so that things work out for the categorical variables
#We will separate it later 
test['loss'] = -1
merged = pd.concat([train, test])
print merged.shape

(313864, 132)


In [2]:
#This is the function that does the cleaning
def extract_mat(df, cat, cont, factorize=False):
    arrs = []
    for col in cont:
        arrs.append(np.array(df[col]))    
    for col in cat:
        if factorize: arrs.append(np.array(pd.factorize(df[col], sort=True)[0])
        else: arrs.append(pd.get_dummies(df[col]).as_matrix().T)
    return np.vstack(arrs).T
    

In [13]:
#ONLY USE THIS DATASET for trees-based models, like random forest
#Pick the columns we want to use in the dataset
cats = cat_cols
conts = cont_cols
#Turn the dataframe into a numpy array
mat = extract_mat(merged, cats, conts, factorize=True)
#extract the training and test datasets from the big dataset
ntrain = len(train)
ntest = len(test)
ntest, ntrain
trainf, testf = mat[:ntrain], mat[ntrain:]
trainl = np.array(train['loss'])
test_ids = np.array(test['id'])
feature_names = conts+cats
utils.save_dataset('data/factorized1.npz', train_features=trainf, test_features=testf, train_labels=trainl, ids=test_ids, feature_names=feature_names)

In [14]:
#We want to make a dataset that doesn't have lots of big categorical variables
#So we drop the ones that are very bad
cats_to_drop = ['cat110', 'cat111', 'cat116'] #categorical variables that have a ridiculous number of possibilities
cats = [var for var in cat_cols if var not in cats_to_drop]
#Turn the dataframe into a numpy array
mat = extract_mat(merged, cats, conts, factorize=False)
#extract the training and test datasets from the big dataset
ntrain = len(train)
ntest = len(test)
ntest, ntrain
trainf, testf = mat[:ntrain], mat[ntrain:]
trainl = np.array(train['loss'])
test_ids = np.array(test['id'])
feature_names = conts+cats
utils.save_dataset('data/no_big_cats1.npz', train_features=trainf, test_features=testf, train_labels=trainl, ids=test_ids, feature_names=feature_names)